In [1]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import spei

In [2]:
# sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

DATABASE_PATH = "../database/database.db"

In [3]:
conn = sqlite3.connect(DATABASE_PATH)

data = pd.read_sql(sql='SELECT * FROM ground_pet_monthly', con=conn)
data['date'] = pd.to_datetime(data['date'])

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13455 entries, 0 to 13454
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   year               13455 non-null  int64         
 1   month              13455 non-null  int64         
 2   date               13455 non-null  datetime64[ns]
 3   region_id          13455 non-null  object        
 4   region_name        13455 non-null  object        
 5   station_id         13455 non-null  object        
 6   station_name       13455 non-null  object        
 7   lat                13455 non-null  float64       
 8   lon                13455 non-null  float64       
 9   station_elevation  13455 non-null  float64       
 10  tmax               5790 non-null   float64       
 11  tmax_count         13455 non-null  int64         
 12  tmin               5791 non-null   float64       
 13  tmin_count         13455 non-null  int64         
 14  tm    

In [7]:
geoinfo

,region_id,region_name,station_name,station_id,lat,lon,station_elevation
0,MASA,Mazandaran,Alasht,99361,36.07,52.84,1805.0
1,MASA,Mazandaran,Amol,99309,36.48,52.47,23.7
2,MASA,Mazandaran,Babolsar,40736,36.70,52.64,-21.0
3,MASA,Mazandaran,Baladeh,99357,36.20,51.80,2120.0
4,MASA,Mazandaran,Bandar-E-Amirabad,99306,36.86,53.39,-20.0
5,MASA,Mazandaran,Galugah,99299,36.74,53.84,-10.0
6,MASA,Mazandaran,Gharakhil,40737,36.49,52.11,14.7
7,MASA,Mazandaran,Kiyasar,40760,36.25,53.55,1294.3
8,MASA,Mazandaran,Kojur,99348,36.39,51.73,1550.0
9,MASA,Mazandaran,Nowshahr,40734,36.66,51.47,-20.9


In [5]:
data_si = data\
    .set_index(['date'])[['station_id', 'rrr24', 'Hargreaves']]

data_si['PE_Hargreaves'] = data_si['rrr24'] - data_si['Hargreaves']

timescale = [1, 3, 6, 9, 12, 15, 18, 21, 24]
i = 1

for ts in timescale:
    df_spi = data_si\
        .groupby(by='station_id')\
        .apply(
            lambda x: spei.spi(
                series=x.rrr24,
                dist=scs.gamma,
                prob_zero=True,
                timescale=ts
            ),
            include_groups=False
        )\
        .reset_index()

    df_spi = df_spi.rename(columns={0: f'SPI_{ts}'})

    if i == 1:
        results = df_spi.copy()
    else:
        results = results.merge(df_spi, on=['station_id', 'date'], how='outer')
    
    df_spei = data_si\
        .groupby(by='station_id')\
        .apply(
            lambda x: spei.spei(
                series=x.PE_Hargreaves,
                dist=scs.fisk,
                prob_zero=True,
                timescale=ts
            ),
            include_groups=False
        )\
        .reset_index()

    df_spei = df_spei.rename(columns={0: f'SPEI_{ts}'})
    
    results = results.merge(df_spei, on=['station_id', 'date'], how='outer')

    i += 1


In [6]:
results

,station_id,date,SPI_1,SPEI_1,SPI_3,SPEI_3,SPI_6,SPEI_6,SPI_9,SPEI_9,SPI_12,SPEI_12,SPI_15,SPEI_15,SPI_18,SPEI_18,SPI_21,SPEI_21,SPI_24,SPEI_24
0,40732,1955-08-31,0.875580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40732,1955-09-30,-0.710605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40732,1955-10-31,-2.418247,-2.097089,-1.729801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40732,1955-11-30,0.846035,0.873020,-0.965064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40732,1955-12-31,-2.667820,-2.302879,-1.139453,-1.290653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5786,99361,2025-05-31,-0.311355,-1.562478,-1.337019,-1.704118,0.286601,-0.433779,0.086396,-0.423194,1.325022,0.206961,0.672943,0.506520,0.883633,0.404525,0.264668,0.364270,0.922135,0.399333
5787,99361,2025-06-30,0.932501,0.678559,-0.304217,-0.853576,0.296904,-0.461051,0.022691,-0.491132,0.418412,0.054976,1.625650,1.120867,1.179523,0.746497,1.056493,0.512614,0.883772,0.284401
5788,99361,2025-07-31,-1.163898,-1.491715,-0.432106,-1.108185,-0.411342,-1.414089,-0.279727,-0.918796,0.078722,-0.358843,1.331262,0.750507,1.107030,0.575682,0.840477,0.216690,0.766412,0.132190
5789,99361,2025-08-31,-0.572522,-0.887478,-0.406638,-0.833320,-1.157216,-1.534368,-0.025431,-0.555976,-0.104749,-0.635258,1.067880,0.461562,0.096805,0.200201,0.759995,0.139107,0.801729,0.167565


In [8]:
results = results.merge(geoinfo, on='station_id', how='left')
results

,station_id,date,SPI_1,SPEI_1,SPI_3,SPEI_3,SPI_6,SPEI_6,SPI_9,SPEI_9,...,SPI_21,SPEI_21,SPI_24,SPEI_24,region_id,region_name,station_name,lat,lon,station_elevation
0,40732,1955-08-31,0.875580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MASA,Mazandaran,Ramsar,36.90,50.68,-20.0
1,40732,1955-09-30,-0.710605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MASA,Mazandaran,Ramsar,36.90,50.68,-20.0
2,40732,1955-10-31,-2.418247,-2.097089,-1.729801,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MASA,Mazandaran,Ramsar,36.90,50.68,-20.0
3,40732,1955-11-30,0.846035,0.873020,-0.965064,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MASA,Mazandaran,Ramsar,36.90,50.68,-20.0
4,40732,1955-12-31,-2.667820,-2.302879,-1.139453,-1.290653,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MASA,Mazandaran,Ramsar,36.90,50.68,-20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5786,99361,2025-05-31,-0.311355,-1.562478,-1.337019,-1.704118,0.286601,-0.433779,0.086396,-0.423194,...,0.264668,0.364270,0.922135,0.399333,MASA,Mazandaran,Alasht,36.07,52.84,1805.0
5787,99361,2025-06-30,0.932501,0.678559,-0.304217,-0.853576,0.296904,-0.461051,0.022691,-0.491132,...,1.056493,0.512614,0.883772,0.284401,MASA,Mazandaran,Alasht,36.07,52.84,1805.0
5788,99361,2025-07-31,-1.163898,-1.491715,-0.432106,-1.108185,-0.411342,-1.414089,-0.279727,-0.918796,...,0.840477,0.216690,0.766412,0.132190,MASA,Mazandaran,Alasht,36.07,52.84,1805.0
5789,99361,2025-08-31,-0.572522,-0.887478,-0.406638,-0.833320,-1.157216,-1.534368,-0.025431,-0.555976,...,0.759995,0.139107,0.801729,0.167565,MASA,Mazandaran,Alasht,36.07,52.84,1805.0


In [9]:
conn = sqlite3.connect(DATABASE_PATH)

results.to_sql('ground_di_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()